### Notebook to get Jay's labels where he only made one correction, should be able to merge with Jaaba prediction to create good full ground truth labels

In [1]:
from pathlib import Path
from tqdm import tqdm
import shutil
import os
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [2]:
columns = ["mat_path", "hand_labels"]
df = pd.DataFrame(columns=columns)

In [3]:
df

,mat_path,hand_labels


In [4]:
mat_dir = Path("/data/caitlin/mat_files/")

In [5]:
def check_all_behaviors(ethogram: np.ndarray):
    count = 0
    for row in ethogram:
        if row.any():
            count += 1
    if count == 1:
        return True
    else:
        return False

In [6]:
mat_paths = sorted(mat_dir.glob('*'))
len(mat_paths)

696

In [7]:
mat_errors = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data"]["trial"].shape[0]
        
    except KeyError:
        mat_errors.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data'].dtype.names.index(f'{b}_labl_label')
            row = m['data'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [8]:
for mat in tqdm(mat_paths):
    hand_labels = get_hand_labels(str(mat))
    if hand_labels:
        df.loc[len(df.index)] = [mat.stem, hand_labels] 

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5548.02it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 5315.10it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5563.38it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5566.61it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5495.23it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5536.67it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5457.16it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 5482.13it/s]

100%|███████████████████████████████████████████████████

In [9]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170314,"{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170328,"{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
434,M336_20210614,"{'M336_20210614_v020': [[0, 0, 0, 0, 0, 0, 0, ..."
435,M336_20210615,"{'M336_20210615_v003': [[0, 0, 0, 0, 0, 0, 0, ..."
436,M336_20210619,"{'M336_20210619_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
437,M336_20210622,"{'M336_20210622_v001': [[0, 0, 0, 0, 0, 0, 0, ..."


In [10]:
mat_errors

['/data/caitlin/mat_files/M236_20170817.mat',
 '/data/caitlin/mat_files/M236_20170818.mat',
 '/data/caitlin/mat_files/M238_20170717.mat',
 '/data/caitlin/mat_files/M238_20170719.mat',
 '/data/caitlin/mat_files/M238_20170724.mat',
 '/data/caitlin/mat_files/M238_20170725.mat',
 '/data/caitlin/mat_files/M238_20170726.mat',
 '/data/caitlin/mat_files/M238_20170727.mat',
 '/data/caitlin/mat_files/M240_20170724FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170725FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170727FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170728FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170801FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170802FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170807FinalChecked.mat',
 '/data/caitlin/mat_files/M240_20170808FinalChecked.mat']

In [11]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

2562

In [13]:
df.to_hdf(path_or_buf="/data/caitlin/exactly1_hand_labels.hdf", key="df")

/tmp/ipykernel_202287/21155089.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/exactly1_hand_labels.hdf", key="df")


In [14]:
df = pd.read_hdf("/data/caitlin/exactly1_hand_labels.hdf")

In [15]:
mat_errors2 = list()
def get_hand_labels(mat_path): 
    hand_labels = dict()

    m = loadmat(mat_path)
    try:
        num_trials = m["data1"]["trial"].shape[0]
        
    except KeyError:
        mat_errors2.append(mat_path)
        return

    for i in tqdm(range(num_trials)):
        mat_trial_index = np.argwhere(m["data1"]["trial"].ravel() == (i + 1))
        # Trial not found in JAABA data
        if mat_trial_index.size == 0:
            continue
        
        try:
            mat_trial_index = mat_trial_index.item()
        except ValueError:
            continue

        behaviors = sorted([b.split('_')[0] for b in m['data1'].dtype.names if 'scores' in b])
        
        if len(behaviors) != 6:
            continue

        all_behaviors = [
            "Lift",
            "Handopen",
            "Grab",
            "Sup",
            "Atmouth",
            "Chew"
        ]

        sorted_behaviors = [b for b in all_behaviors if b in behaviors]

        ethograms = []

        for b in sorted_behaviors:
            behavior_index = m['data1'].dtype.names.index(f'{b}_labl_label')
            row = m['data1'][mat_trial_index][0][behavior_index]
            row[row == -1] = 0
            ethograms.append(row)

        sorted_behaviors = [b.lower() for b in sorted_behaviors]

        ethogram = np.hstack(ethograms).T

        if check_all_behaviors(ethogram):
            hand_labels[m["data1"]["exp"][i][0][0]] = ethogram

    return hand_labels

In [16]:
for mat in tqdm(mat_errors):
    hand_labels = get_hand_labels(Path(mat))
    if hand_labels:
        df.loc[len(df.index)] = [Path(mat).stem, hand_labels] 

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5668.07it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 81/81 [00:00<00:00, 5509.78it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5616.27it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5581.71it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 5306.12it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5605.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 5577.16it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.73it/s]


In [17]:
mat_errors2

[]

In [18]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170314,"{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170328,"{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
445,M240_20170724FinalChecked,"{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ..."
446,M240_20170725FinalChecked,"{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ..."
447,M240_20170727FinalChecked,"{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ..."
448,M240_20170802FinalChecked,"{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
df.to_hdf(path_or_buf="/data/caitlin/exactly1_hand_labels.hdf", key="df")

/tmp/ipykernel_202287/21155089.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['mat_path', 'hand_labels'], dtype='object')]

  df.to_hdf(path_or_buf="/data/caitlin/exactly1_hand_labels.hdf", key="df")


In [20]:
df = pd.read_hdf("/data/caitlin/exactly1_hand_labels.hdf")

In [21]:
df

,mat_path,hand_labels
0,M232_20170306,"{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170314,"{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170328,"{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
445,M240_20170724FinalChecked,"{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ..."
446,M240_20170725FinalChecked,"{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ..."
447,M240_20170727FinalChecked,"{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ..."
448,M240_20170802FinalChecked,"{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
count = 0
for row in df.iterrows():
    count += len(row[1]["hand_labels"].keys())
count

2586